### Random Forest Modeling for Bunning 2019 Human aging dataset



#### Step 1: Data preprocessing. 
We will combind the categorical data in sheet 2 into the numercial data and use them together for regression modeling

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

First, import the transformed data and contrapose it.

In [14]:
df = pd.read_excel("../transformed_data.xlsx", sheet_name=0, index_col=0)
df = df.T
print(df)

Metabolite name  L-Alanine  L-Aspartic acid  L-Asparagine  \
P1               -1.846172         0.484126      0.021412   
P2               -0.443329        -0.877494     -0.061673   
P3               -0.134794         0.206897      0.104678   
P4                1.068429        -0.146876     -0.031757   
P5                0.459775         0.530521      0.103970   
...                    ...              ...           ...   
P264             -1.348079         0.226587      0.110009   
P265             -0.694988         0.232135      0.170186   
P266             -1.474038         0.165482      0.056151   
P267             -1.353003         0.642928     -0.023424   
P268             -0.916607        -0.639974      0.368144   

Metabolite name  Glucosamine|Galactosamine  Hydroxyhippurate  L-Carnitine  \
P1                                0.861255         -0.173662     1.018054   
P2                                0.264052         -0.454759    -3.766291   
P3                                0.

Concatenate the categorical data from sheet 2 into the dataframe.

In [17]:
table2df = pd.read_excel("../acel13073-sup-0002-tables1-s3.xlsx", sheet_name=1, header=[0,1])
table2df.head()

KeyError: 'Age'